\begin{equation}
I(t) = I(0) + (\beta(S(0)+I(0))-\gamma)
\int_0^t I(\tau)\, d\tau - \beta \int_0^t I(\tau)^2\, d\tau - \beta \gamma \int_0^t I(\tau)
\left(
\int_0^{\tau} I(s)\, ds
\right) d\tau
\end{equation}

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from scipy.optimize import least_squares
import pandas as pd

def SIR(y, t, beta, gamma):
    """
    :param y: initial condition
    :param t: time
    :param beta: para
    :param gamma: para
    :return: the SIR model ODEs
    """
    S, I, R = y
    dS = - beta * S * I
    dI = beta * S * I - gamma * I
    dR = gamma * I
    return [dS, dI, dR]

# initial condition
S0 = 9999
I0 = 1
R0 = 0
y = S0, I0, R0

# true parameters
beta = 0.00001
gamma = 0.01

# time
t = np.linspace(0, 500, 1000)

# simulate SIR
solution = scipy.integrate.odeint(SIR, [S0, I0, R0], t, args=(beta, gamma))
S, I, R = solution.T

I_data = I + np.random.normal(0, 100, size=I.shape)

I_int = np.array([simpson(I_data[:i+1], t[:i+1]) for i in range(len(t))])
I_int2 = np.array([simpson(I_data[:i+1]**2, t[:i+1]) for i in range(len(t))])
int_double = 1/2 * (I_int ** 2)

pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', None)         # Auto width
pd.set_option('display.max_colwidth', None)  # Full content in each column

## Residual with separate parameters $\beta$, $\gamma$ and $S0$

In [ ]:
"""
Residual with separate parameters beta, gamma and S0
"""

def residual(paras, I_data, I_int, I_int2, int_double, I0):
    beta, gamma, S0 = paras
    I_hat = I0 + ((beta * S0 + beta * I0) - gamma) * I_int - beta * I_int2 - beta * gamma * int_double
    return I_data - I_hat

# Initial value for parameters
beta_initial_list = [0.01, 0.001, 0.0001]
gamma_initial_list = [1, 0.1, 0.05]
S0_initial_list = [100, 1000, 8000]

results = []

for beta_initial in beta_initial_list:
    for gamma_initial in gamma_initial_list:
        for S0_initial in S0_initial_list:
            x0 = [beta_initial, gamma_initial, S0_initial]
            res = least_squares(residual, x0, args=(I_data, I_int, I_int2, int_double, I0))

            results.append({
                "beta0": beta_initial,
                "gamma0": gamma_initial,
                "S00": S0_initial,
                "beta_hat": res.x[0],
                "gamma_hat": res.x[1],
                "S0_hat": res.x[2],
                "cost": res.cost,
                "success": res.success
            })

In [ ]:
df = pd.DataFrame(results)
table = df[['beta0', 'gamma0', 'S00', 'beta_hat', 'gamma_hat', 'S0_hat', 'cost']]

print(f"True beta {beta}; True gamma {gamma}; True S0 {S0}")
print(table)

In [ ]:
df["beta_error"] = df["beta_hat"] - beta
df["gamma_error"] = df["gamma_hat"] - gamma
df["S0_error"] = df["S0_hat"] - S0

table = df[['beta0', 'gamma0', 'S00','beta_hat', 'gamma_hat', 'S0_hat', 'beta_error', 'gamma_error', 'S0_error', 'cost']]
print(table)
latex_code = table.to_latex(index=False, float_format="%.5e")
print(latex_code)

## Residual with parameters and their products

### $\alpha = \beta S0$

In [ ]:
def residual(paras, I_data, I_int, I_int2, int_double, I0):
    beta, gamma, alpha = paras
    I_hat = I0 + ((alpha + beta * I0) - gamma) * I_int - beta * I_int2 - beta * gamma * int_double
    return I_data - I_hat

# Initial value for parameters
beta_initial_list = np.array([0.01, 0.001, 0.0001])
gamma_initial_list = np.array([1, 0.1, 0.05])
S0_initial_list = np.array([100, 1000, 8000])
alpha_initial_list = beta_initial_list * S0_initial_list

results = []

for beta_initial in beta_initial_list:
    for gamma_initial in gamma_initial_list:
        for alpha_initial in alpha_initial_list:
            x0 = [beta_initial, gamma_initial, alpha_initial]
            res = least_squares(residual, x0, args=(I_data, I_int, I_int2, int_double, I0))

            results.append({
                "beta0": beta_initial,
                "gamma0": gamma_initial,
                "alpha0": alpha_initial,
                "beta_hat": res.x[0],
                "gamma_hat": res.x[1],
                "S0_hat": res.x[2] / res.x[0],
                "cost": res.cost,
                "success": res.success
            })

In [ ]:
df = pd.DataFrame(results)
table = df[['beta0', 'gamma0', 'alpha0', 'beta_hat', 'gamma_hat', 'S0_hat', 'cost']]

print(f"True beta {beta}; True gamma {gamma}; True S0 {S0}")
print(table)

In [ ]:
df["beta_error"] = df["beta_hat"] - beta
df["gamma_error"] = df["gamma_hat"] - gamma
df["S0_error"] = df["S0_hat"] - S0

table = df[['beta0', 'gamma0', 'alpha0','beta_hat', 'gamma_hat', 'S0_hat', 'beta_error', 'gamma_error', 'S0_error', 'cost']]
print(table)
latex_code = table.to_latex(index=False, float_format="%.5e")
print(latex_code)